# Data Science Capstone

Notebook for the *Data Science Capstone Course*. Part of the <b>Data Science Professional Certificate</b> in Coursera. Offered by IBM <br>
<b>Notebook Owner:</b> Adriana Cortés Buelvas

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!


## Segmenting and Clustering Neighborhoods in Toronto

### Exploring and Scraping : Assignment Part 1


In this first part of the Capstone Project I am going to explore and cluster some information of Toronto Neighborhoods. For this, I will need *'to scrape the following Wikipedia page, https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M, in order to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe'*, as indicaded on the course.

To do that, I am going to read the data set in the link as a Dataframe using Pandas' method `.read_html`. Since the web age holds more than one dataframe, it is necessary to select the first one by using the index `[0]` <br>
Then, I will drop all the rows that has a value of 'Not assigned' on the **Borough** column. 

In [6]:
df = pd.read_html('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')[0]
df = df[df.Borough != 'Not assigned']

From the DataFrame above, notice that, those Postal Codes that corresponds to more than one neighbourhood, will indicate all of them in the same row and separated by a coma. For example, look at M6A: in the column **Neighborhood** it has Lawrence Manor and Lawrence Heights. 

In [4]:
for x in df.Neighborhood:
    if x == 'Not assigned':
        x == df.Borough
print(df.head())

  Postal Code           Borough                                 Neighborhood
2         M3A        North York                                    Parkwoods
3         M4A        North York                             Victoria Village
4         M5A  Downtown Toronto                    Regent Park, Harbourfront
5         M6A        North York             Lawrence Manor, Lawrence Heights
6         M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government


Note: There should be a more efficient way for the last code using a lambda function. However, I couldn't some upt with it. If you know how, please leave it on the feedback :)

Finally, I print the shape to now the number of rows does the Dataframe have

In [5]:
df.shape

(103, 3)